# Initialization

In [1]:
from logging import ERROR
import os
from pathlib import Path
import signal
from subprocess import CalledProcessError
import subprocess
import time
import sys
import traceback

import mpbn

# trap-mvn init
import trapspace

import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    raise AssertionError()
    raise KeyboardInterrupt()
    
signal.signal(signal.SIGALRM, handler)

<Handlers.SIG_DFL: 0>

In [2]:
# compute trap spaces of multi-valued networks using trap-mvn
def bench_trap_mvn(models, model_type=".sbml", timeout=120, number=5, max_output=1000, computation="min", semantics="general", fixmethod="1"):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                tspaces = trapspace.compute_trap_spaces(model + model_type, max_output=max_output, computation=computation, semantics=semantics, fixmethod=fixmethod)
                nb_tspaces = len(list(tspaces))
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError, AssertionError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [3]:
# compute minimal trap spaces or fixed points of Boolean networks using mpbn
def bench_mpbn(models, timeout=120, number=5, max_output=1000, computation="fix"):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                sbml_file = model + ".sbml"
                bnet_file = model + ".bnet"
                #os.system( "java -jar bioLQM.jar " + sbml_file + " " + bnet_file)
                subprocess.call(['java', '-jar', 'bioLQM.jar', sbml_file, bnet_file], stdout=subprocess.DEVNULL)
                mbn = mpbn.load(model + ".bnet")
                if computation == "fix":
                    tspaces = mbn.fixedpoints(limit=max_output)
                else:
                    tspaces = mbn.attractors(limit=max_output)
                tspaces = list(tspaces)
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
                os.unlink(bnet_file)
            except AssertionError:
                if i == 0:
                    print("Non-monotonic")
            except (TimeOutException, CalledProcessError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [4]:
# compute trap spaces of Boolean networks using trappist
def bench_trappist(models, timeout=120, number=5, max_output=1000, method="asp", computation="min", fixmethod="1"):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                sbml_file = model + ".sbml"
                bnet_file = model + ".bnet"
                #os.system( "java -jar bioLQM.jar " + sbml_file + " " + bnet_file)
                subprocess.call(['java', '-jar', 'bioLQM.jar', sbml_file, bnet_file], stdout=subprocess.DEVNULL)
                tspaces = trappist.compute_trap_spaces(model + ".bnet", max_output=max_output, method=method, computation=computation, fixmethod=fixmethod)
                nb_tspaces = len(list(tspaces))
                end = time.perf_counter()
                total_time += end - start
                successes += 1
                os.unlink(bnet_file)
            except (TimeOutException, CalledProcessError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [5]:
def process_output_an_asp(output: str):
    lines = output.split("\\n")
    n_fixed_points = 0
    
    for line in lines:
        if "Models       :" in line:
            tmp = (line.split(":"))[1]
            
            if tmp.endswith("+"):
                tmp = tmp[:-1]
                
            n_fixed_points = int(tmp)
            break
            
    return n_fixed_points

In [6]:
# compute trap spaces of Boolean networks using trappist
def bench_an_asp(models, timeout=120, number=5, max_output=1000):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                sbml_file = model + ".sbml"
                an_file = model + ".an"
                lp_file = model + ".lp"
                #os.system( "java -jar bioLQM.jar " + sbml_file + " " + bnet_file)
                subprocess.call(['java', '-jar', 'bioLQM.jar', sbml_file, an_file], stdout=subprocess.DEVNULL)
                subprocess.call(['python', 'AN2asp.py', an_file, lp_file], stdout=subprocess.DEVNULL)
                
                output = subprocess.run(['clingo', str(max_output), 'fixed-points.lp', '-q', lp_file], capture_output=True)
                n_fixed_points = process_output_an_asp(str(output))
                
                end = time.perf_counter()
                total_time += end - start
                successes += 1
                os.unlink(an_file)
                os.unlink(lp_file)
            except (TimeOutException, CalledProcessError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{n_fixed_points} fixed points {total_time/number:.2f}\n===")
            #print(str(output) + "\n")
        else:
            print("DNF\n===")

## Multi-valued networks in .sbml

In [21]:
models = [str(Path().absolute() / "sbml" / name[:-5]) for name in os.listdir("sbml") if name.endswith(".sbml")]

In [20]:
bench_trap_mvn(models, max_output=1000, timeout=120, computation="fix", semantics="unitary", fixmethod="1")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 trap spaces 0.29
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 trap spaces 2.20
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.09
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 trap spaces 0.09
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
18 trap spaces 0.02
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 trap spaces 0.08
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
0 trap spaces 0.82
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
1000 trap spaces 0.80
===
/home/giang-trinh/g

In [22]:
bench_trap_mvn(models, max_output=1000, timeout=120, computation="fix", semantics="unitary", fixmethod="2")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 trap spaces 0.18
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 trap spaces 1.66
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.06
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 trap spaces 0.08
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
18 trap spaces 0.02
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 trap spaces 0.06
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
0 trap spaces 0.28
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
1000 trap spaces 0.45
===
/home/giang-trinh/g

In [10]:
bench_trap_mvn(models, max_output=1000, timeout=120, computation="min", semantics="general")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 trap spaces 0.31
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 trap spaces 4.23
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.04
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 trap spaces 0.12
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
19 trap spaces 0.02
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 trap spaces 0.07
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
1 trap spaces 0.90
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
1000 trap spaces 0.83
===
/home/giang-trinh/g

In [11]:
bench_trap_mvn(models, max_output=1000, timeout=120, computation="min", semantics="unitary")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 trap spaces 0.28
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 trap spaces 2.27
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.04
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 trap spaces 0.11
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
19 trap spaces 0.02
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 trap spaces 0.07
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
1 trap spaces 0.83
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
1000 trap spaces 0.84
===
/home/giang-trinh/g

In [12]:
bench_trap_mvn(models, max_output=1000, timeout=120, computation="max", semantics="general")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
63 trap spaces 0.10
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
78 trap spaces 3.68
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.05
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
9 trap spaces 0.13
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
8 trap spaces 0.01
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
22 trap spaces 0.03
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
12 trap spaces 0.94
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
75 trap spaces 0.17
===
/home/giang-trinh/git_repo

In [13]:
bench_trap_mvn(models, max_output=1000, timeout=120, computation="max", semantics="unitary")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
63 trap spaces 0.09
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
62 trap spaces 1.71
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.05
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
8 trap spaces 0.11
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
8 trap spaces 0.02
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
20 trap spaces 0.05
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
10 trap spaces 0.81
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
75 trap spaces 0.15
===
/home/giang-trinh/git_repo

In [14]:
bench_mpbn(models, max_output=1000, timeout=120, computation="fix")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 trap spaces 0.87
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 trap spaces 6.03
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.44
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 trap spaces 6.90
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
18 trap spaces 0.36
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 trap spaces 0.67
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
Non-monotonic
DNF
==

In [15]:
bench_mpbn(models, max_output=1000, timeout=120, computation="min")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 trap spaces 0.83
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 trap spaces 5.68
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 trap spaces 0.47
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 trap spaces 6.37
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
19 trap spaces 0.37
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 trap spaces 0.65
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
Timeout
Timeout
Timeout
Timeout
Timeout
DNF
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
Non-monotonic
DNF
==

In [16]:
bench_trappist(models, timeout=120, max_output=1000, method="asp", computation="min")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
1000 trap spaces 0.59
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
1000 trap spaces 1.40
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
4 trap spaces 0.47
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary


In [17]:
bench_trappist(models, timeout=120, max_output=1000, method="asp", computation="fix", fixmethod="1")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
1000 trap spaces 0.55
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
1000 trap spaces 1.42
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
4 trap spaces 0.44
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary


In [18]:
bench_trappist(models, timeout=120, max_output=1000, method="asp", computation="fix", fixmethod="2")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
# nodes = 103, # transitions = 403
1000 trap spaces 0.51
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
# nodes = 300, # transitions = 2528
1000 trap spaces 1.19
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
# nodes = 96, # transitions = 363
4 trap spaces 0.46
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary


In [19]:
bench_an_asp(models, timeout=120, max_output=1000)

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
1000 fixed points 0.43
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
1000 fixed points 0.54
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
4 fixed points 0.49
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
1 fixed points 0.48
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
18 fixed points 0.38
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
654 fixed points 0.37
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/146_BUDDING-YEAST-FAURE-2009
0 fixed points 0.79
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/195-CTLA4-PD1-checkpoint-inhibitors
1000 fixed points 0.49
===
/home/giang

In [7]:
models = [str(Path().absolute() / "bma" / name[:-5]) for name in os.listdir("bma") if name.endswith(".json")]

In [8]:
bench_trap_mvn(models, model_type=".json", max_output=1000, number=1, timeout=120, computation="min", semantics="unitary")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/bma/Leukaemia.free-inputs
729 trap spaces 0.57
===


In [8]:
bench_trappist(models, timeout=120, max_output=1000, number=1, method="asp", computation="fix", fixmethod="2")

/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/157_CONTROL-OF-TH-DIFFERENTATION
# nodes = 103, # transitions = 403
1000 trap spaces 0.60
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/SkinModel.free-inputs
# nodes = 300, # transitions = 2528
1000 trap spaces 1.13
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/161_DIFFERENTIATION-OF-MONOCYTES
# nodes = 96, # transitions = 363
4 trap spaces 0.40
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/197-anterior-posterior-boundary
# nodes = 56, # transitions = 504
1 trap spaces 0.45
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/178-mast-cell-activation
# nodes = 48, # transitions = 114
18 trap spaces 0.35
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/sbml/175_SEA-URCHIN
# nodes = 41, # transitions = 135
654 trap spaces 0.41
===
/home/giang-trinh/git_repo/multi-valued-logical-models/trapspace/s